In [1]:
import os
import sys
import numpy as np
import pandas as pd

sys.path.append("../python")

import api


In [2]:
# Apply an embedding to a single text

mytext = "local zoning ordinances"
myembedding = api.get_embedding(mytext)
print(type(myembedding))
print(len(myembedding))
myembedding[0:10]

<class 'numpy.ndarray'>
1536


array([-0.00358684,  0.03781258,  0.00146078,  0.01892056,  0.00328898,
        0.0225734 ,  0.00680983, -0.05068313, -0.02325831, -0.00781936])

In [3]:
# Get embeddings on an entire dataframe

df = pd.read_csv("../../intermediate_data/chapter_data.csv")

df['embedding'] = df['Chapter Text'].apply(
    lambda x: api.get_embedding(x)
)

In [4]:
# Save CSV file with embeddings
# Note: For larger operations we may want to use a vector store
df.to_pickle("../../intermediate_data/chapter_data_embedded.pkl")

# Load CSV file with embeddings
df = pd.read_pickle("../../intermediate_data/chapter_data_embedded.pkl")

df.head()

,Chapter ID,Chapter Text,embedding
0,JD_C1,CHAPTER I | GENERAL PROVISIONS AND ZONING,"[0.03730521351099014, 0.021854771301150322, 0...."
1,JD_Ch1a,CHAPTER 1A | CITY OF LOS ANGELES ZONING CODE,"[0.03449486196041107, 0.039579860866069794, 0...."
2,JD_C2,"CHAPTER II* | LICENSES, PERMITS, BUSINESS REGU...","[0.03710675612092018, 0.04733029752969742, 0.0..."
3,JD_C3,CHAPTER III | PUBLIC HEALTH CODE,"[0.03463377431035042, 0.01646377518773079, 0.0..."
4,JD_C4,CHAPTER IV | PUBLIC WELFARE,"[0.04702747240662575, 0.0075818439945578575, 0..."
